### Monthly Air Pollution in Malaysia (2017-2022)
Source: https://open.dosm.gov.my/data-catalogue/air_pollution?visual=concentration

In [8]:
import pandas as pd

URL_DATA = 'https://storage.data.gov.my/environment/air_pollution.parquet'

df = pd.read_parquet(URL_DATA)
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])

df

,date,pollutant,concentration
0,2017-01-01,CO,0.5610
1,2017-02-01,CO,0.5300
2,2017-03-01,CO,0.5890
3,2017-04-01,CO,0.6620
4,2017-05-01,CO,NaN
...,...,...,...
427,2022-08-01,SO2,0.0012
428,2022-09-01,SO2,0.0012
429,2022-10-01,SO2,0.0012
430,2022-11-01,SO2,0.0012


We will be standardizing the data against WHO's 2021 air quality guidelines: <br>
https://iris.who.int/server/api/core/bitstreams/551b515e-2a32-4e1a-a58c-cdaecd395b19/content

- PM2.5(µg/m³) < 15 μg/m³
- PM10(µg/m³) < 45 μg/m³
- O3(ppm to µg/m³) < 60 µg/m³. Conversion factors for ozone: at 20 °C and 1013 hPa, 1 part per million (ppm) = 1.9957 mg/m3 and 1 mg/m3 = 0.5011 ppm
- NO2(ppm to µg/m³) < 25 μg/m3. Conversion factors: at 20 °C and 1013 hPa, 1 ppm = 1.914 mg/m 3 and 1 mg/m3 = 0.523 ppm
- SO2(ppm to µg/m³) < 40 μg/m3. Conversion factors: at 20 °C and 1013 hPa, 1 ppm = 2660 μg/m3 and 1 mg/m3 = 0.3759 ppm.
- CO(ppm to µg/m³) < 4000 µg/m³. Conversion factors: at 20 °C and 1013 hPa, 1 ppm = 1.165 mg/m3 and 1 mg/m3 = 0.858 ppm.

In [9]:
# convert all rows to standard units (µg/m³)
df.loc[df['pollutant'] == 'O3', 'concentration'] /= 0.5011
df.loc[df['pollutant'] == 'NO2', 'concentration'] /= 0.523
df.loc[df['pollutant'] == 'SO2', 'concentration'] /= 0.3759
df.loc[df['pollutant'] == 'CO', 'concentration'] /= 0.858

df.loc[df['pollutant'] == 'O3', 'concentration'] *= 1000
df.loc[df['pollutant'] == 'NO2', 'concentration'] *= 1000
df.loc[df['pollutant'] == 'SO2', 'concentration'] *= 1000
df.loc[df['pollutant'] == 'CO', 'concentration'] *= 1000

df

,date,pollutant,concentration
0,2017-01-01,CO,653.846154
1,2017-02-01,CO,617.715618
2,2017-03-01,CO,686.480186
3,2017-04-01,CO,771.561772
4,2017-05-01,CO,NaN
...,...,...,...
427,2022-08-01,SO2,3.192338
428,2022-09-01,SO2,3.192338
429,2022-10-01,SO2,3.192338
430,2022-11-01,SO2,3.192338


In [15]:
# aggregate all pollutant concentrations into yearly averages
df_yearly = df.groupby([df['date'].dt.year.rename('year'), 'pollutant']).mean(numeric_only=True).reset_index()

In [17]:
# calculate percentage of concentration against WHO guidelines
WHO_GUIDELINES = {
    'PM 2.5': 15,
    'PM 10': 45,
    'O3': 60,
    'NO2': 25,
    'SO2': 40,
	'CO': 4000
}

df_yearly['percentage'] = df_yearly.apply(lambda row: (row['concentration'] / WHO_GUIDELINES[row['pollutant']]) * 100, axis=1)
df_yearly

,year,pollutant,concentration,percentage
0,2017,CO,752.797203,18.819930
1,2017,NO2,15.066922,60.267686
2,2017,O3,32.727998,54.546664
3,2017,PM 10,30.047000,66.771111
4,2017,PM 2.5,NaN,NaN
5,2017,SO2,4.522479,11.306198
6,2018,CO,759.518260,18.987956
7,2018,NO2,14.627151,58.508604
8,2018,O3,35.422071,59.036786
9,2018,PM 10,24.919167,55.375926


In [19]:
df_yearly.to_csv('../data/pollutants/air_pollution_yearly.csv', index=False)

### Air Pollution Index in Malaysia (2000 - 2022)
historical api data - https://www.kaggle.com/datasets/ynshung/malaysia-air-pollution-index <br>
lat and long data - https://eqms.doe.gov.my/APIMS/main

In [2]:
import pandas as pd

df = pd.read_csv('../data/api/APIMS-final.csv')
df

,Time,Alor Gajah,Alor Setar,Balik Pulau,Balok Baru Kuantan,Bandaraya Melaka,Banting,Batu Muda,Batu Pahat,Besut,...,Sibu,Sri Aman,Sungai Petani,Taiping,Tanah Merah,Tangkak,Tanjung Malim,Tasek Ipoh,Tawau,Temerloh
0,2005-10-01 11:00,NaN,NaN,NaN,55.0,59.0,61.0,35.0,NaN,NaN,...,42.0,NaN,33.0,41.0,44.0,NaN,34.0,33.0,51.0,NaN
1,2005-10-01 17:00,NaN,NaN,NaN,57.0,52.0,70.0,40.0,NaN,NaN,...,39.0,NaN,30.0,35.0,46.0,NaN,31.0,30.0,53.0,NaN
2,2005-10-02 11:00,NaN,NaN,NaN,50.0,49.0,60.0,41.0,NaN,NaN,...,34.0,NaN,29.0,44.0,44.0,NaN,28.0,27.0,52.0,NaN
3,2005-10-02 17:00,NaN,NaN,NaN,50.0,46.0,58.0,48.0,NaN,NaN,...,33.0,NaN,32.0,42.0,43.0,NaN,30.0,30.0,49.0,NaN
4,2005-10-03 11:00,NaN,NaN,NaN,57.0,34.0,53.0,30.0,NaN,NaN,...,35.0,NaN,35.0,53.0,34.0,NaN,24.0,28.0,53.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71454,2022-05-07 07:00,31.0,37.0,54.0,49.0,37.0,41.0,48.0,37.0,53.0,...,32.0,64.0,55.0,42.0,49.0,32.0,35.0,59.0,39.0,53.0
71455,2022-05-07 08:00,30.0,37.0,54.0,50.0,37.0,41.0,48.0,37.0,53.0,...,32.0,63.0,54.0,43.0,49.0,31.0,36.0,59.0,38.0,53.0
71456,2022-05-07 09:00,30.0,37.0,54.0,51.0,38.0,43.0,45.0,36.0,54.0,...,31.0,63.0,55.0,43.0,48.0,31.0,35.0,58.0,38.0,53.0
71457,2022-05-07 10:00,30.0,36.0,53.0,50.0,38.0,43.0,43.0,37.0,54.0,...,30.0,63.0,55.0,41.0,50.0,32.0,35.0,58.0,38.0,53.0


Check null and NaN values

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71459 entries, 0 to 71458
Data columns (total 66 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Time                    71459 non-null  object 
 1   Alor Gajah              43892 non-null  float64
 2   Alor Setar              70269 non-null  float64
 3   Balik Pulau             44064 non-null  float64
 4   Balok Baru Kuantan      70970 non-null  float64
 5   Bandaraya Melaka        69223 non-null  float64
 6   Banting                 60992 non-null  float64
 7   Batu Muda               69606 non-null  float64
 8   Batu Pahat              43260 non-null  float64
 9   Besut                   43770 non-null  float64
 10  Bintulu                 70258 non-null  float64
 11  Bukit Rambai            70284 non-null  float64
 12  Cheras                  67360 non-null  float64
 13  Indera Mahkota Kuantan  69544 non-null  float64
 14  Jerantut                69741 non-null

We will aggregate all the data entries to a yearly basis

In [4]:
# convert 'Time' to datetime with inconsistent formats
df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

# get month and year from 'Time'
df['month'] = df['Time'].dt.strftime('%m')
df['year'] = df['Time'].dt.strftime('%Y')

In [5]:
# get a list of all other columns except 'Time' and 'month'
columns_to_average = [col for col in df.columns if col not in ['Time', 'month', 'year']]
columns_to_average

['Alor Gajah',
 'Alor Setar',
 'Balik Pulau',
 'Balok Baru Kuantan',
 'Bandaraya Melaka',
 'Banting',
 'Batu Muda',
 'Batu Pahat',
 'Besut',
 'Bintulu',
 'Bukit Rambai',
 'Cheras',
 'Indera Mahkota Kuantan',
 'Jerantut',
 'Kangar',
 'Kapit',
 'Kemaman',
 'Keningau',
 'Kimanis',
 'Klang',
 'Kluang',
 'Kota Bharu',
 'Kota Kinabalu',
 'Kota Tinggi',
 'Kuala Selangor',
 'Kuala Terengganu',
 'Kuching',
 'Kulim',
 'Labuan',
 'Langkawi',
 'Larkin',
 'Limbang',
 'Minden',
 'Miri',
 'Muar',
 'Mukah',
 'Nilai',
 'Paka',
 'Pasir Gudang',
 'Pegoh Ipoh',
 'Pengerang',
 'Petaling Jaya',
 'Port Dickson',
 'Putrajaya',
 'Rompin',
 'Samalaju',
 'Samarahan',
 'Sandakan',
 'Sarikei',
 'Seberang Jaya',
 'Seberang Perai',
 'Segamat',
 'Seremban',
 'Seri Manjung',
 'Shah Alam',
 'Sibu',
 'Sri Aman',
 'Sungai Petani',
 'Taiping',
 'Tanah Merah',
 'Tangkak',
 'Tanjung Malim',
 'Tasek Ipoh',
 'Tawau',
 'Temerloh']

In [18]:
# aggregate via average by month and year
df_monthly = df.groupby(['year', 'month'])[columns_to_average].mean().reset_index()
df_monthly

,year,month,Alor Gajah,Alor Setar,Balik Pulau,Balok Baru Kuantan,Bandaraya Melaka,Banting,Batu Muda,Batu Pahat,...,Sibu,Sri Aman,Sungai Petani,Taiping,Tanah Merah,Tangkak,Tanjung Malim,Tasek Ipoh,Tawau,Temerloh
0,2005,10,NaN,26.625000,NaN,52.241935,42.338710,45.790323,37.919355,NaN,...,37.500000,31.277778,40.564516,40.870968,34.661290,NaN,33.306452,37.725806,51.903226,NaN
1,2005,11,NaN,26.224138,NaN,46.983333,44.183333,41.566667,31.633333,NaN,...,37.633333,29.611111,40.200000,34.833333,35.232143,NaN,26.250000,34.633333,49.716667,NaN
2,2005,12,NaN,26.689655,NaN,47.645161,46.098361,46.290323,38.333333,NaN,...,35.032258,29.400000,37.516667,39.316667,33.600000,NaN,25.032258,33.483333,48.129032,NaN
3,2006,01,NaN,40.145161,NaN,50.383333,51.112903,48.596774,40.316667,NaN,...,37.306452,36.274194,47.483871,47.096774,38.822581,NaN,31.177419,41.290323,53.083333,NaN
4,2006,02,NaN,34.125000,NaN,48.446429,42.185185,40.428571,34.357143,NaN,...,36.821429,36.464286,42.214286,47.148148,33.553571,NaN,26.763636,40.339286,54.803571,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2022,01,39.059219,50.699865,54.562584,45.119785,42.300135,65.788978,59.358008,36.597577,...,34.391655,31.864065,57.063257,57.191117,61.666218,42.248656,48.106326,58.358008,47.639300,44.133244
178,2022,02,37.120715,42.467958,49.385991,44.570790,42.481371,57.101190,52.061103,35.332817,...,37.904620,32.186289,49.041729,43.073025,47.578241,34.900149,37.692996,53.357675,45.283159,34.284650
179,2022,03,45.460581,42.395161,47.111559,39.541667,48.166667,59.250000,55.031680,40.241047,...,36.629032,33.723118,49.845430,46.173387,54.502035,48.166891,46.592742,55.943978,43.442204,41.012179
180,2022,04,50.647222,44.808333,46.815278,44.240278,50.213889,62.686695,58.136111,44.284286,...,34.615385,40.247222,53.006944,46.852778,50.579167,49.922222,46.588889,56.673611,43.504286,41.769444


In [19]:
# add a 'date' column for easier merging later
df_monthly['date'] = pd.to_datetime(df_monthly['year'] + '-' + df_monthly['month'] + '-01')

# round all columns to 2 decimal places except 'year', 'month', and 'date'
df_monthly[columns_to_average] = df_monthly[columns_to_average].round(2)

# remove 'year' and 'month' columns
df_monthly = df_monthly.drop(columns=['year', 'month'])

df_monthly

,Alor Gajah,Alor Setar,Balik Pulau,Balok Baru Kuantan,Bandaraya Melaka,Banting,Batu Muda,Batu Pahat,Besut,Bintulu,...,Sri Aman,Sungai Petani,Taiping,Tanah Merah,Tangkak,Tanjung Malim,Tasek Ipoh,Tawau,Temerloh,date
0,NaN,26.62,NaN,52.24,42.34,45.79,37.92,NaN,NaN,39.88,...,31.28,40.56,40.87,34.66,NaN,33.31,37.73,51.90,NaN,2005-10-01
1,NaN,26.22,NaN,46.98,44.18,41.57,31.63,NaN,NaN,43.92,...,29.61,40.20,34.83,35.23,NaN,26.25,34.63,49.72,NaN,2005-11-01
2,NaN,26.69,NaN,47.65,46.10,46.29,38.33,NaN,NaN,44.89,...,29.40,37.52,39.32,33.60,NaN,25.03,33.48,48.13,NaN,2005-12-01
3,NaN,40.15,NaN,50.38,51.11,48.60,40.32,NaN,NaN,40.55,...,36.27,47.48,47.10,38.82,NaN,31.18,41.29,53.08,NaN,2006-01-01
4,NaN,34.12,NaN,48.45,42.19,40.43,34.36,NaN,NaN,43.11,...,36.46,42.21,47.15,33.55,NaN,26.76,40.34,54.80,NaN,2006-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,39.06,50.70,54.56,45.12,42.30,65.79,59.36,36.60,51.98,44.12,...,31.86,57.06,57.19,61.67,42.25,48.11,58.36,47.64,44.13,2022-01-01
178,37.12,42.47,49.39,44.57,42.48,57.10,52.06,35.33,36.06,46.55,...,32.19,49.04,43.07,47.58,34.90,37.69,53.36,45.28,34.28,2022-02-01
179,45.46,42.40,47.11,39.54,48.17,59.25,55.03,40.24,49.56,51.51,...,33.72,49.85,46.17,54.50,48.17,46.59,55.94,43.44,41.01,2022-03-01
180,50.65,44.81,46.82,44.24,50.21,62.69,58.14,44.28,48.88,52.06,...,40.25,53.01,46.85,50.58,49.92,46.59,56.67,43.50,41.77,2022-04-01


In [20]:
# melt the dataframe with 'id_vars' as 'date'
df_monthly = df_monthly.melt(id_vars=['date'], var_name='station', value_name='aqi')
df_monthly

,date,station,aqi
0,2005-10-01,Alor Gajah,NaN
1,2005-11-01,Alor Gajah,NaN
2,2005-12-01,Alor Gajah,NaN
3,2006-01-01,Alor Gajah,NaN
4,2006-02-01,Alor Gajah,NaN
...,...,...,...
11825,2022-01-01,Temerloh,44.13
11826,2022-02-01,Temerloh,34.28
11827,2022-03-01,Temerloh,41.01
11828,2022-04-01,Temerloh,41.77


In [21]:
# remove rows with NaN values in 'aqi' column
df_monthly = df_monthly.dropna(subset=['aqi'])
df_monthly

,date,station,aqi
120,2017-04-01,Alor Gajah,21.68
121,2017-05-01,Alor Gajah,27.63
122,2017-06-01,Alor Gajah,30.98
123,2017-07-01,Alor Gajah,30.20
124,2017-08-01,Alor Gajah,26.30
...,...,...,...
11825,2022-01-01,Temerloh,44.13
11826,2022-02-01,Temerloh,34.28
11827,2022-03-01,Temerloh,41.01
11828,2022-04-01,Temerloh,41.77


In [22]:
df_monthly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9938 entries, 120 to 11829
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     9938 non-null   datetime64[ns]
 1   station  9938 non-null   object        
 2   aqi      9938 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 310.6+ KB


In [23]:
# load another dataset with lan/long, state and country info
df_locations = pd.read_csv('../data/api/malaysia_aqi.csv')
df_locations = df_locations.drop(columns=['uid', 'aqi', 'time'])
df_locations.rename(columns={'station_location': 'station'}, inplace=True)
df_locations

,lat,lon,station,state,country
0,3.177084,113.041091,Bintulu,Sarawak,Malaysia
1,2.926645,103.419198,Rompin,Pahang,Malaysia
2,4.200344,100.663358,Seri Manjung,Perak,Malaysia
3,5.401688,100.589680,Kulim Hi-Tech,Kedah,Malaysia
4,2.014498,112.927360,Kapit,Sarawak,Malaysia
...,...,...,...,...,...
60,2.914816,101.690050,Putrajaya,Wilayah Persekutuan,Malaysia
61,5.339317,116.163658,Keningau,Sabah,Malaysia
62,3.960644,103.382158,Kuantan,Pahang,Malaysia
63,4.262121,103.425778,Kemaman,Terengganu,Malaysia


In [24]:
# perform left join on df_monthly with df_locations on 'station' column
df_monthly = df_monthly.merge(df_locations, on='station', how='left')
df_monthly

,date,station,aqi,lat,lon,state,country
0,2017-04-01,Alor Gajah,21.68,2.370925,102.224592,Melaka,Malaysia
1,2017-05-01,Alor Gajah,27.63,2.370925,102.224592,Melaka,Malaysia
2,2017-06-01,Alor Gajah,30.98,2.370925,102.224592,Melaka,Malaysia
3,2017-07-01,Alor Gajah,30.20,2.370925,102.224592,Melaka,Malaysia
4,2017-08-01,Alor Gajah,26.30,2.370925,102.224592,Melaka,Malaysia
...,...,...,...,...,...,...,...
10115,2022-01-01,Temerloh,44.13,3.471603,102.376406,Pahang,Malaysia
10116,2022-02-01,Temerloh,34.28,3.471603,102.376406,Pahang,Malaysia
10117,2022-03-01,Temerloh,41.01,3.471603,102.376406,Pahang,Malaysia
10118,2022-04-01,Temerloh,41.77,3.471603,102.376406,Pahang,Malaysia


In [25]:
df_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10120 entries, 0 to 10119
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     10120 non-null  datetime64[ns]
 1   station  10120 non-null  object        
 2   aqi      10120 non-null  float64       
 3   lat      7975 non-null   float64       
 4   lon      7975 non-null   float64       
 5   state    7975 non-null   object        
 6   country  7975 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 553.6+ KB


In [26]:
# show stations with NaN values
df_monthly[df_monthly['lat'].isna()]['station'].unique()

array(['Balok Baru Kuantan', 'Batu Muda', 'Cheras',
       'Indera Mahkota Kuantan', 'Klang', 'Kulim', 'Larkin', 'Muar',
       'Pegoh Ipoh', 'Seberang Jaya', 'Seberang Perai', 'Sungai Petani',
       'Tasek Ipoh'], dtype=object)

In [27]:
df_missing_locations = pd.DataFrame({
    'station': [
        'Balok Baru Kuantan',
        'Batu Muda',
        'Cheras',
        'Indera Mahkota Kuantan',
        'Klang',
        'Kulim',
        'Larkin',
        'Muar',
        'Pegoh Ipoh',
        'Seberang Jaya',
        'Seberang Perai',
        'Sungai Petani',
        'Tasek Ipoh'
    ],
    'state': [
        'Pahang',
        'Wilayah Persekutuan',
        'Wilayah Persekutuan',
        'Pahang',
        'Selangor',
        'Kedah',
        'Johor',
        'Johor',
        'Perak',
        'Pulau Pinang',
        'Pulau Pinang',
        'Kedah',
        'Perak'
    ],
    'country': ['Malaysia'] * 13,
    'lat': [
        3.96,      # Balok Baru Kuantan (provided)
        3.21,      # Batu Muda (provided)
        3.21,      # Cheras
        3.82,      # Indera Mahkota Kuantan
        3.01,      # Klang
        3.03,      # Kulim
        1.49,      # Larkin
        2.04,      # Muar
        4.60,      # Pegoh Ipoh
        5.40,      # Seberang Jaya
        5.38,      # Seberang Perai
        5.65,      # Sungai Petani
        4.63       # Tasek Ipoh
    ],
    'lon': [
        103.38,    # Balok Baru Kuantan (provided)
        101.68,    # Batu Muda (provided)
        101.27,    # Cheras
        103.30,    # Indera Mahkota Kuantan
        101.41,    # Klang
        101.57,    # Kulim
        103.73,    # Larkin
        102.57,    # Muar
        101.08,    # Pegoh Ipoh
        100.39,    # Seberang Jaya
        100.40,    # Seberang Perai
        100.48,    # Sungai Petani
        101.12     # Tasek Ipoh
    ]
})

In [28]:
# join again to fill in missing locations
df_monthly = df_monthly.merge(df_missing_locations, on='station', how='left', suffixes=('', '_missing'))

# fill in missing lat/lon/state/country from df_missing_locations
df_monthly['lat'] = df_monthly['lat'].fillna(df_monthly['lat_missing'])
df_monthly['lon'] = df_monthly['lon'].fillna(df_monthly['lon_missing'])
df_monthly['state'] = df_monthly['state'].fillna(df_monthly['state_missing'])
df_monthly['country'] = df_monthly['country'].fillna(df_monthly['country_missing'])
df_monthly = df_monthly.drop(columns=['lat_missing', 'lon_missing', 'state_missing', 'country_missing'])
df_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10120 entries, 0 to 10119
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     10120 non-null  datetime64[ns]
 1   station  10120 non-null  object        
 2   aqi      10120 non-null  float64       
 3   lat      10120 non-null  float64       
 4   lon      10120 non-null  float64       
 5   state    10120 non-null  object        
 6   country  10120 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 553.6+ KB


In [1]:
# save to csv
df_monthly.to_csv('../data/api/APIMS-monthly.csv', index=False)

NameError: name 'df_monthly' is not defined

Now, we are interested in average aqi for entire malaysia

In [4]:
import pandas as pd

df = pd.read_csv('../data/api/APIMS-monthly.csv')

# drop irrelevant columns
df = df.drop(columns=['state', 'country', 'lat', 'lon'])

# aggregate aqi by finding the mean aqi for that month
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df_monthly_avg = df.groupby(['year', 'month'])['aqi'].mean().reset_index()
df_monthly_avg


,year,month,aqi
0,2005,10,38.920000
1,2005,11,37.176600
2,2005,12,38.352200
3,2006,1,42.936800
4,2006,2,41.205800
...,...,...,...
177,2022,1,46.988462
178,2022,2,41.586154
179,2022,3,44.162308
180,2022,4,47.054615


In [5]:
# convert back to date
df_monthly_avg['date'] = pd.to_datetime(df_monthly_avg['year'].astype(str) + '-' + df_monthly_avg['month'].astype(str) + '-01')
df_monthly_avg = df_monthly_avg.drop(columns=['year', 'month'])
df_monthly_avg

,aqi,date
0,38.920000,2005-10-01
1,37.176600,2005-11-01
2,38.352200,2005-12-01
3,42.936800,2006-01-01
4,41.205800,2006-02-01
...,...,...
177,46.988462,2022-01-01
178,41.586154,2022-02-01
179,44.162308,2022-03-01
180,47.054615,2022-04-01


In [7]:
# fill in missing months with NaN values
all_months = pd.date_range(start=df_monthly_avg['date'].min(), end=df_monthly_avg['date'].max(), freq='MS')
df_monthly_avg = df_monthly_avg.set_index('date').reindex(all_months).rename_axis('date').reset_index()
df_monthly_avg

,date,aqi
0,2005-10-01,38.920000
1,2005-11-01,37.176600
2,2005-12-01,38.352200
3,2006-01-01,42.936800
4,2006-02-01,41.205800
...,...,...
195,2022-01-01,46.988462
196,2022-02-01,41.586154
197,2022-03-01,44.162308
198,2022-04-01,47.054615


In [8]:
# save to csv
df_monthly_avg.to_csv('../data/api/APIMS-monthly-avg.csv', index=False)

This code here gets average aqi for every state

In [1]:
import pandas as pd

df = pd.read_csv('../data/api/APIMS-monthly.csv')

# drop irrelevant columns
df = df.drop(columns=['station', 'country', 'lat', 'lon'])

# aggregate aqi by finding the mean aqi for that state and month
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df_monthly_avg = df.groupby(['year', 'month', 'state'])['aqi'].mean().reset_index()
df_monthly_avg


,year,month,state,aqi
0,2005,10,Johor,43.887500
1,2005,10,Kedah,34.403333
2,2005,10,Kelantan,33.305000
3,2005,10,Melaka,51.095000
4,2005,10,Negeri Sembilan,46.540000
...,...,...,...,...
2531,2022,5,Sabah,32.808000
2532,2022,5,Sarawak,37.909167
2533,2022,5,Selangor,58.834000
2534,2022,5,Terengganu,44.267500


In [2]:
df_monthly_avg['date'] = pd.to_datetime(df_monthly_avg['year'].astype(str) + '-' + df_monthly_avg['month'].astype(str) + '-01')
df_monthly_avg = df_monthly_avg.drop(columns=['year', 'month'])
df_monthly_avg.to_csv('../data/api/APIMS-monthly-avg-by-state.csv', index=False)

In [19]:
import pandas as pd

df1 = pd.read_csv('../data/api/APIMS-monthly-avg-by-state.csv')
df2 = pd.read_csv('../data/api/APIMS-monthly-avg.csv')

# add a new 'Malaysia' state to df1 with the average aqi from df2
df2['state'] = 'Malaysia'
df1 = pd.concat([df1, df2], ignore_index=True)
df1

,state,aqi,date
0,Johor,43.887500,2005-10-01
1,Kedah,34.403333,2005-10-01
2,Kelantan,33.305000,2005-10-01
3,Melaka,51.095000,2005-10-01
4,Negeri Sembilan,46.540000,2005-10-01
...,...,...,...
3201,Malaysia,46.988462,2022-01-01
3202,Malaysia,41.586154,2022-02-01
3203,Malaysia,44.162308,2022-03-01
3204,Malaysia,47.054615,2022-04-01


In [20]:
# find dates with null values from df2
df2[df2['aqi'].isna()]

# add date and aqi with null values to all states in df1
missing_dates = df2[df2['aqi'].isna()]['date'].unique()
missing_rows = pd.DataFrame([(date, state, None) for date in missing_dates for state in df1['state'].unique()], columns=['date', 'state', 'aqi'])
df1 = pd.concat([df1, missing_rows], ignore_index=True)
df1[df1['aqi'].isna()]

# for all null cells, impute with "null"
df1['aqi'] = df1['aqi'].fillna('null')
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476 entries, 0 to 3475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   3476 non-null   object
 1   aqi     3476 non-null   object
 2   date    3476 non-null   object
dtypes: object(3)
memory usage: 81.6+ KB


/tmp/ipykernel_26929/1291427880.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df1 = pd.concat([df1, missing_rows], ignore_index=True)


In [21]:
# make all values 2 decimal places except 'state' and 'date'
df1['aqi'] = pd.to_numeric(df1['aqi'], errors='coerce').round(2)
df1

,state,aqi,date
0,Johor,43.89,2005-10-01
1,Kedah,34.40,2005-10-01
2,Kelantan,33.30,2005-10-01
3,Melaka,51.10,2005-10-01
4,Negeri Sembilan,46.54,2005-10-01
...,...,...,...
3471,Sarawak,NaN,2016-12-01
3472,Selangor,NaN,2016-12-01
3473,Terengganu,NaN,2016-12-01
3474,Wilayah Persekutuan,NaN,2016-12-01


In [23]:
df1.to_csv('../data/api/APIMS-monthly-avg-by-state.csv', index=False)

### Average Daily Traffic (2015 - 2024) - https://www.mot.gov.my/en/media/annual-report/yearly-statistic

Extract tables from PDF using Camelot

In [6]:
import camelot
import pandas as pd

def extract_tables_from_pdf(pdf_path, page_range='1', flavor='lattice'):
    """
    Extract tables from PDF and return as a single DataFrame.
    
    Parameters:
    -----------
    pdf_path : str
        Path to the PDF file
    page_range : str
        Page numbers to extract. Examples:
        - '1' for single page
        - '1-3' for range
        - '1,3,5' for specific pages
        - '1-3,5,7-9' for mixed
        - 'all' for all pages
    flavor : str
        'lattice' for tables with borders (default, more accurate)
        'stream' for tables without borders
    
    Returns:
    --------
    pandas.DataFrame or list of DataFrames
        Extracted table(s)
    """
    
    # Extract tables from PDF
    tables = camelot.read_pdf(pdf_path, pages=page_range, flavor=flavor)
    
    print(f"Found {len(tables)} table(s)")
    
    if len(tables) == 0:
        print("No tables found!")
        return None
    
    # Display accuracy for each table
    for i, table in enumerate(tables):
        print(f"Table {i+1} - Accuracy: {table.accuracy:.2f}%, Shape: {table.df.shape}")
    
    # If only one table, return as DataFrame
    if len(tables) == 1:
        return tables[0].df
    
    # If multiple tables, return as list
    return [table.df for table in tables]


# Example usage:
# Replace 'your_file.pdf' with your actual PDF path
dfs = extract_tables_from_pdf('../docs/Transport Statistics Malaysia 2024.pdf', page_range='41-44', flavor='stream')

Found 4 table(s)
Table 1 - Accuracy: 99.80%, Shape: (33, 13)
Table 2 - Accuracy: 99.18%, Shape: (31, 13)
Table 3 - Accuracy: 98.44%, Shape: (27, 13)
Table 4 - Accuracy: 99.63%, Shape: (22, 13)


Code to clean dataframes

In [7]:
def clean_data(df):
    # make a copy
    df = df.copy()
    
    # find start row where first column is 'BIL'
    start_idx = df[df.iloc[:, 0] == 'BIL'].index[0]
    df = df.iloc[start_idx:].reset_index(drop=True)
    
    # find last complete row (all values non-null and non-empty)
    mask = df.notna().all(axis=1) & (df.astype(str).apply(lambda x: x.str.strip() != '').all(axis=1))
    end_idx = mask[::-1].idxmax()  # find last True value
    df = df.iloc[:end_idx + 1].reset_index(drop=True)
    
    # merge first two rows as header
    new_header = (df.iloc[0].astype(str) + ' ' + df.iloc[1].astype(str)).str.strip()
    df = df.iloc[3:].reset_index(drop=True)  # skip first 3 rows (header + 1 data row)
    df.columns = new_header
    
    # identify state header rows (empty BIL column)
    bil_col = df.columns[0]
    is_state_row = df[bil_col].isna() | (df[bil_col].astype(str).str.strip() == '')
    
    # forward fill state values
    df['State'] = None
    df.loc[is_state_row, 'State'] = df.loc[is_state_row, '2016'].str.lower().str.title().str.strip()
    df['State'] = df['State'].ffill()
    
    # remove state header rows
    df = df[~is_state_row].reset_index(drop=True)
    
    return df

In [8]:
# clean each dataframe
cleaned_dfs = [clean_data(df) for df in dfs]

# Merge in wide format (easier to align on common columns like State, Station)
merged_df = pd.concat(cleaned_dfs, ignore_index=True)

# remove BIL column and use natural index
merged_df = merged_df.drop(columns=[merged_df.columns[0]])

# Then melt to long format
melted_df = merged_df.melt(
    id_vars=['STESEN', 'LOKASI', 'State'],  # adjust column names
    var_name='Year',
    value_name='ADT',
    value_vars=['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']  # adjust year columns
)

# rename STESEN to Station and LOKASI to Location
melted_df = melted_df.rename(columns={'STESEN': 'Station', 'LOKASI': 'Location'})

# remove station = WR103
melted_df = melted_df[melted_df['Station'] != 'WR 103']

# convert ADT to numeric, removing commas
melted_df['ADT'] = pd.to_numeric(melted_df['ADT'].str.replace(',', ''), errors='coerce')

# assign a rank column based on ADT within each year
melted_df['rank'] = melted_df.groupby('Year')['ADT'].rank(ascending=False, method='first')

melted_df

,Station,Location,State,Year,ADT,rank
0,AR 101,Ipoh-Tanjung Malim (Slim River Toll house),Perak,2015,16736,41.0
1,AR 204,Ipoh-Lumut,Perak,2015,26494,22.0
2,AR 301,Ipoh-Kampar,Perak,2015,30592,17.0
3,AR 303,Sitiawan-Gopeng,Perak,2015,78136,6.0
4,AR 501,Ipoh-Kuala Kangsar,Perak,2015,19895,32.0
...,...,...,...,...,...,...
785,SR 401,Bintulu-Sibu,Sarawak,2024,7204,64.0
786,SR 402,Bintulu-Miri,Sarawak,2024,11931,47.0
787,SR 403,Miri-Bintulu (before junction to Airport),Sarawak,2024,10163,54.0
788,SR 503,Limbang-Brunei Border (Sg.Padaruan),Sarawak,2024,11895,49.0


Save to csv

In [9]:
melted_df.to_csv('../data/traffic/adt_data.csv', index=False)

### Road geojson data - https://overpass-turbo.eu/

We first merge all our roads

In [11]:
import json
import glob
import os

all_features = []

# Process each GeoJSON file
for filepath in glob.glob("../data/map/top10roads/*.geojson"):
    # Extract road name from filename
    road_name = os.path.basename(filepath).replace('.geojson', '')
    
    # Read the GeoJSON
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    # Add road_name and collect features
    for feature in data['features']:
        if 'properties' not in feature:
            feature['properties'] = {}
        feature['properties']['Location'] = road_name
        all_features.append(feature)

# Create combined GeoJSON
combined = {
    "type": "FeatureCollection",
    "features": all_features
}

# Save combined file
with open("../data/map/all_roads.geojson", "w") as f:
    json.dump(combined, f, indent=2)

print(f"Done! Combined {len(all_features)} features into all_roads.geojson")

Done! Combined 3065 features into all_roads.geojson


Then, we merge the csv with the roads

In [ ]:
import geopandas as gpd
import pandas as pd

# 1. Load your road GeoJSON data
gdf = gpd.read_file('../data/map/all_roads.geojson')

# 2. Load your traffic CSV
traffic_df = pd.read_csv('../data/traffic/adt_data.csv')  

# 3. Merge the data
# The 'on' parameter should be the common identifier between datasets
# Common examples: 'road_id', 'segment_id', 'street_name', etc.
merged_data = gdf.merge(traffic_df, on='Location', how='left')

In [13]:
merged_data.to_file('../data/map/merged_traffic_roads.geojson', driver='GeoJSON')

### Malaysia map AQI choropleth map

In [39]:
import geopandas as gpd
import pandas as pd

# 1. Load your states topojson data
gdf = gpd.read_file('../data/map/geoBoundaries.topojson', layer='MYSADM1gbOpen')

gdf['shapeName'] = gdf['shapeName'].replace(
    {
        'Malacca': 'Melaka',
        'Penang': 'Pulau Pinang',
        'Kuala Lumpur': 'Wilayah Persekutuan',
        'Putrajaya': 'Wilayah Persekutuan',
        'Labuan': 'Wilayah Persekutuan'
    },
)

# 2. Load your state-level AQI CSV
aqi_df = pd.read_csv('../data/api/APIMS-monthly-avg-by-state.csv')


# 3. Perform left join on gdf with aqi_df on 'shapeName' and 'state' columns
merged_data = gdf.merge(aqi_df, left_on='shapeName', right_on='state', how='left')
merged_df = merged_data.drop(columns=['id', 'shapeName', 'shapeID', 'shapeGroup', 'shapeType', 'geometry'])
merged_df = pd.concat([merged_df, aqi_df[(aqi_df['state'] == 'Malaysia')]])

# 4. fillna as "null"
merged_df['aqi'] = merged_df['aqi'].fillna('null')

# 4. Save as csv
merged_df.to_csv('../data/api/APIMS-monthly-avg-by-state.csv', index=False)

In [42]:
df = pd.read_csv('../data/api/APIMS-monthly-avg-by-state.csv')
df.fillna('null', inplace=True)
df.to_csv('../data/api/APIMS-monthly-avg-by-state.csv', index=False)

/tmp/ipykernel_45277/95315388.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'null' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('null', inplace=True)
